In [24]:
%load_ext autoreload

%autoreload 2

import gauleg as gl 
import sympy as sp 
import numpy as np 
import pandas as pd 
import math

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Finding Lm Ln Polynomials 


In [59]:
def legendre_polynomial(n):
    if n == 0:
        return sp.Lambda(y, 1)
    elif n == 1:
        return sp.Lambda(y, y)
    else:
        P_n_minus_1 = legendre_polynomial(n-1)
        P_n_minus_2 = legendre_polynomial(n-2)
        return sp.Lambda(y, ((2*n - 1) * y * P_n_minus_1(y) - (n - 1) * P_n_minus_2(y)) / n)

# Example usage:
n = 3
Ln = legendre_polynomial(n)
Ln


Lambda(y, 5*y*(3*y**2/2 - 1/2)/3 - 2*y/3)

## Computing Anm using GL Quadrature --> finding A 

In [78]:
n = [0,1,2]
m = [0,1,2,3]

def Anm(n, m):
    y = sp.symbols('y')
    Ln = legendre_polynomial(n)
    Lm = legendre_polynomial(m)
    return y * Ln(y) * Lm(y)

def Anm_gauleg(xi, ci, b , a, n2, n, m):
    sum = 0 
    Anm_expr = Anm(n,m)
    for i in range(n2):
        y_value = (0.5*(((b-a)*xi[i])+(b+a)))
        prod = ci[i]* Anm_expr.subs(y, y_value)
        sum = sum + prod 
    return sum 

def A_matrix(n,m):
    A_matrix = sp.zeros(len(n),len(m))
    n2 = math.floor((len(n)+len(m))/2) + 1
    xi = gl.gauleg(n2)[0] # satisfying dop of GL for p+1 is 2p + 1
    ci = gl.gauleg(n2)[1]
    for ni in n:
        for mi in m:
            A_matrix[ni ,mi] = Anm_gauleg(xi= xi, ci = ci, b = 1, a = -1, n2 = n2, n = ni, m = mi)
    return A_matrix

anm = Anm(n = 2,m = 2)
anm


y*(3*y**2/2 - 1/2)**2

## Define A(x,y) 

In [75]:
def a(x, y):
    x = sp.symbols('x')
    y = sp.symbols('y')
    return 5 * x + 1 + y * sp.sin(x)

def a0(x):
    x = sp.symbols('x')
    return 5 * x + 1 

def a1(x):
    x = sp.symbols('x')
    return sp.sin(x)


## Define (S1) matrix and (S0) matrix


In [83]:
def GL(xi, ci, y, x, n2, func):
	sum = 0 
	for i in range(n2):
		prod = ci[i]*func(0.5*(((y-x)*xi[i])+(y+x)))
		sum = sum + prod 
	return (y-x)/2 * (sum)


def phiij(numofnodes, i, j, l, xlist):
	finalsum = 0
	for k in range(numofnodes-1):

		x = xlist[k]
		y = xlist[k+1]
	 
		if k == i :
			d_phi_i = 2**l 
		elif k == i + 1 :
			d_phi_i = -2**l 
		else: 
			d_phi_i = 0 
 
		if k == j : 
			d_phi_j = 2**l 
		elif k == j + 1 :
			d_phi_j = -2**l 
		else:
			d_phi_j = 0 

		n2 = 5 
		xi = gl.gauleg(n2)[0]
		ci = gl.gauleg(n2)[1]
		onegl = GL(xi = xi, ci = ci, y = y, x = x, n2 = 5, func = a1) * d_phi_i *d_phi_j
		finalsum = finalsum + onegl

	return finalsum

def S1(listi, listj):
    S1 = sp.zeros(len(listi), len(listj))
    l = 2
    for i in listi :
        for j in listj:
            S1[i,j] = gl.phiij(numofnodes=2**l +1, i = i, j = j, xlist = gl.listi(a= 0,b =1, h = 2 **(-l), n1 = 2 **l + 1), l = l)
    return S1

S1(listi = [0,1,2], listj=[0,1,2])


Matrix([
[  17.958679009754, -9.46127775712435,                 0],
[-9.46127775712435,  19.7955768453892, -10.3342990882648],
[                0, -10.3342990882648,  21.3964840963557]])